# Cube-Net ConvLSTM (Cross) Training Environment
This notebooks contains the setup and training of the convolutional LSTM network for the Cube-Net project. More specifically, this notebook contains the training environment for the cross solving network.

### cube Python Bindings

The Cube class is used to provide python bindings to the rust program 'cube' which is used to generate a dataset of scrambled cubes, both as a Tensor representation as well as their associated scrambles.

In [1]:
from cube_bindings import Cube

cube = Cube()


Compiling the solution_verifier Rust program in release mode...
Compilation successful.


    Finished release [optimized] target(s) in 0.23s


### Example
Here is an example of how the python bindings for the rust program are used:

With regards to the training environment for the cross solving model, the follow operations are available: 
- cube.generate_data() generate a batch of scrambled cube Tensors and the scrambles that produced them
- cube.is_solved() and cube.is_cross_solved() to check if the cube or the cross is solved
- cube. solved_cross() to determine the solution of the cross

In [2]:
cube_tensors, scrambles = cube.generate_data(batch_size=1, scramble_len=40)

cube_tensors.shape

solved = cube.is_solved(scrambles[0], "")
print(f"Is the cube solved? {'yes' if solved else 'no'} --- scramble: {scrambles[0]}")

cross_solution = cube.solve_cross(scrambles[0])
print(f"Cross solution: {cross_solution}")  

solved = cube.is_solved(scrambles[0], cross_solution)
print(f"Is the cube now solved? {'yes' if solved else 'no'}")

all_moves_so_far = " ".join([scrambles[0], cross_solution])
print(f"All moves so far: {all_moves_so_far}")

print(f"Is the cross now solved? {'yes' if cube.is_cross_solved(all_moves_so_far) else 'no'}")


Is the cube solved? no --- scramble: L U' F' L B D F L R' R U' L D' U' F' B' U' B F' U L U' D D U R D U' R B' B R' F' L R' F' U U' L L'
Cross solution: U U U F F R' U U R'' L L B U U B' U U U R R B R' U U U R B B 
Is the cube now solved? no
All moves so far: L U' F' L B D F L R' R U' L D' U' F' B' U' B F' U L U' D D U R D U' R B' B R' F' L R' F' U U' L L' U U U F F R' U U R'' L L B U U B' U U U R R B R' U U U R B B 
Is the cross now solved? yes


# Setting up the Training Environment

Here I am setting up the training environment for the cross solving model.

The model below accepts data in the following format: (batch_size, time_steps, channels, height, width, depth) 
Where the height, width, depth dimmensions are the rubiks cube tensor representaiton. The channels dimmension is initially a singleton dimmension, but will be exapanded upon convolution. 

The time_steps dimmension is a bit special to this implementation in that it will be increased as the solution to a scrambled cube is found. Previous cube states will be stored in the time_steps dimmension (up to a maximum). 

ConvLSTMClassifier itself is only tasked with determining the best *single move* to make given the current cube state. However, the previous cube states are stored in the time_steps dimmension so that the model is informed of the context of the cube state.

In [3]:
from model_cross import ConvLSTMClassifier, ConvLSTM
import torch
 
# Initialize the ConvLSTM
conv_lstm = ConvLSTM(input_channels=1, hidden_channels=[8, 16, 32, 32, 64], kernel_size=3)

num_output_features = 64 * 5 * 5 * 5  # Replace with the correct size

# Initialize ConvLSTMClassifier
classifier = ConvLSTMClassifier(conv_lstm, num_output_features, num_classes=13)

# Example input and target data for classification
input = torch.randn(1, 5, 1, 5, 5, 5)
output = classifier(input)
prediction = output.argmax(dim=1)

# Print shapes for verification
print('Input size:', input.shape)
print('Output size:', output.shape)
print('Prediction size:', prediction.shape)

Input size: torch.Size([1, 5, 1, 5, 5, 5])
Output size: torch.Size([1, 13])
Prediction size: torch.Size([1])


In [4]:
input = torch.randn(32, 5, 1, 5, 5, 5)

output = conv_lstm(input)
print(f'Conv LSTM output shape: {output.shape}')

classifer_output = classifier(input)
print(f'Classifier output shape: {classifer_output.shape}')

Conv LSTM output shape: torch.Size([32, 64, 5, 5, 5])
Classifier output shape: torch.Size([32, 13])


In [5]:
from train_cross import Trainer, TrainConfig
from early_stopping import EarlyStopping
import torch

config = TrainConfig(
    scramble_len=40,
    epochs=1,
    val_num_batches=10,
    batch_size=32,
    lr=0.001,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    optimizer=torch.optim.Adam,
    early_stopping=EarlyStopping(patience=10)
)


trainer = Trainer(cube, config, classifier)

/home/holindauer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
trainer.train()

cube_states shape: torch.Size([32, 1, 1, 5, 5, 5])
length of max solution: 31
cube_states shape: torch.Size([32, 1, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 2, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 3, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 4, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 5, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 6, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 7, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size([32])
cube_states shape: torch.Size([32, 8, 1, 5, 5, 5])
outputs shape: torch.Size([32, 13])
pred shape: torch.Size(